# **Notebook for Prompts (using Langchain, Outputparser, ...)**

0 [Orchestrator](#Orchestrator)<br>

1 [Database](#database)<br>
1.1 [Database Dev](#database-dev) <br>
1.2 [Database Test](#Database-Test) <br>
1.3 [Database Doc](#Database-Doc) <br>

2 [Backend](#Backend)<br>
2.1 [Backend Dev](#Backend-Dev) <br>
2.2 [Backend Test](#Backend-Test) <br>
2.3 [Backend Doc](#Backend-Doc) <br>

3 [Frontend](#Frontend)<br>
3.1 [Frontend Dev](#Frontend-Dev) <br>
3.2 [Frontend Test](#Frontend-Test) <br>
3.3 [Frontend Doc](#Frontend-Doc) <br>

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

import openai
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

In [2]:
# _ = load_dotenv(find_dotenv()) # read local .env file

# import warnings
# warnings.filterwarnings('ignore')

# llm_model = "gpt-3.5-turbo"

In [3]:
# def get_completion(prompt, model=llm_model):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0, 
#     )
#     return response.choices[0].message["content"]

# get_completion("Hi",)

In [4]:
# chat = ChatOpenAI(temperature=0.0, model=llm_model)
# chat

In [5]:
def read_txt(file_path):
    """
    Reads text from a file located at 'file_path' and returns it as a string.

    :param file_path: Path to the text file.
    :return: String containing the contents of the file.
    """
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except IOError:
        return "Error: File not found or cannot be read."

## **Orchestrator**

In [6]:
user_prompt = """Create a simple web-application that can be used to sign up for a newsletter. I have the following requirements:
A database that stores the necessary information about the subscribers.
An frontend that has a modern design, with a big red headline and short description of my newsletter in text form. 
All the necessary signup fields where someone can enter their data.
A button that when pressed saves the data in the database.
A popup message that confirms the successful entry."""

In [7]:
# prompt components
prompt_txt = read_txt('orchestrator.txt')
orchestrator_schema_1 = ResponseSchema(name="orchestrator_db",
                             description="The tasks for the database")
orchestrator_schema_2 = ResponseSchema(name="orchestrator_be",
                             description="The tasks for the backend")
orchestrator_schema_3 = ResponseSchema(name="orchestrator_fe",
                             description="The tasks for the frontend")

# define ouput parser
response_schemas = [orchestrator_schema_1,orchestrator_schema_2,orchestrator_schema_3]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
orchestrator_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 user_prompt = user_prompt)

In [8]:
orchestrator_prompt_str = orchestrator_prompt[0].content
print(orchestrator_prompt_str)

You are the manager of "The Agentsy" a software company that does full-stack web development for simple webpages. There are three other specialists in your company a database-engineer who develops database schemas, a backend-engineer who develops all the backend functionalities and a frontend engineer who develops the frontend.  You will be provided with a user request for a new website. Based on this information it is your task to create requirements for your engineers, so they can fulfill their job. 

Provide the following information:
Overall task: a very short description what the user wants and what the overall task is
Database: what the database needs to be capable of in general as well as what tables and fields are needed
Backend: what kind of calls to the database need to be made
Frontend: how the frontend should look and what functionalities there are

Do not provide any additional output or comments.
Now this is the request by the user:
Create a simple web-application that ca

## **Database**

### Database Dev

In [9]:
# prompt components
prompt_txt = read_txt('database_dev.txt')
database_dev_schema_1 = ResponseSchema(name="code",
                             description="SQL Statements you wrote for the task.")
orchestrator_db = "Schema for a newsletter signup page." # Example

# define ouput parser
response_schemas = [database_dev_schema_1]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
database_dev_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 orchestrator_db = orchestrator_db)

In [10]:
database_dev_prompt_str = database_dev_prompt[0].content
print(database_dev_prompt_str)

You are a software developer specialized in databases. It is your responsibility to write correct and high quality SQL statements for different projects. You will be provided with several requirements your manager came up with based on some user request. It is now your task to provide the data definition language that fulfill these requirements. You should there for output the necessary SQL statements that create the tables for the database as well as any other SQL statements that are needed. These are the requirements:
Schema for a newsletter signup page.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"code": string  // SQL Statements you wrote for the task.
}
```
Only output this, and put your generated code in the json. Do not output any additional comments.


In [11]:
# this is how a response would be parsed to get values
# response = chat(database_dev_prompt)
# output_dict = output_parser.parse(response.content)
# output_dict["code"]

### Database Test

In [12]:
# prompt components
prompt_txt = read_txt('database_test.txt')
database_test_schema_1 = ResponseSchema(name="revision_needed",
                                        description="Based on your evaluation, are there any errors that need to be fixed? Answer True if yes, False if no.")
database_test_schema_2 = ResponseSchema(name="feedback",
                                        description="Report with your analysis of the code.")

database_code = "No code"

# define ouput parser
response_schemas = [database_test_schema_1, database_test_schema_2]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
database_test_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 database_code = database_code)

In [13]:
database_test_prompt_str = database_test_prompt[0].content
print(database_test_prompt_str)

You are a software developer specialized in testing databases. It is your responsibility to check a database design and SQL statements for any errors. You will be provided with some SQL code. It is your task to validate this code an check for any mistakes such as syntax errors, data type mismatchs, missing or incorrect joins, unhandled NULL values, improperly formed queries, inefficient queries or any other errors you can think of. 
This is the SQL code you have to check:
No code

Based on this code write a report and point out any mistakes or errors you have found. Explain them and suggest how they could be corrected. Only point errors, mistakes or things that are incorrect do not talk about things that are well done. It is possible that there are no errors or bad practices.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"revision_needed": string  // Based on your evaluation, are ther

### Database Doc

In [14]:
# prompt components
prompt_txt = read_txt('database_doc.txt')
database_doc_schema_1 = ResponseSchema(name="documentation",
                                        description="The documentation of the code.")

database_code = "No code"

# define ouput parser
response_schemas = [database_doc_schema_1]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
database_doc_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 database_code = database_code)

In [15]:
database_doc_prompt_str = database_doc_prompt[0].content
print(database_doc_prompt_str)

You are a specialist for writing a documentation for code. It is your responsibility to analyze some given code and write a short description, so that someone else can understand what functionalities there are. You will be provided with some SQL code that is used to create a database. It is your task to write a short documentation that describes what tables are created.
This is the SQL code:
No code

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"documentation": string  // The documentation of the code.
}
```


## **Backend**

### Backend Dev

In [16]:
# prompt components
prompt_txt = read_txt('backend_dev.txt')
backend_dev_schema_1 = ResponseSchema(name="code",
                             description="The code you wrote based on the described requirements.")
backend_dev_schema_2 = ResponseSchema(name="pip_packages",
                             description="The names of the pip packages that need to be installed. In a list.")


orchestrator_be = ""
database_doc = ""

# define ouput parser
response_schemas = [backend_dev_schema_1, backend_dev_schema_2]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
backend_dev_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 orchestrator_be = orchestrator_be,
                                 database_doc = database_doc)

In [17]:
backend_dev_prompt_str = backend_dev_prompt[0].content
print(backend_dev_prompt_str)

You are a software developer specialized in backend systems. It is your responsibility to write the python API calls that enable access to a database. You will be provided with some requirements by your manager and a description of an existing database. It is now your task to provide the code that enables read and write access to this database and fulfills the listed requirements. Use fastapi to do this and only provide the final code block.
These are the requirements by your manager:


This is the description of the database:


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"code": string  // The code you wrote based on the described requirements.
	"pip_packages": string  // The names of the pip packages that need to be installed. In a list.
}
```
Only output this, and put your generated code in the json. Do not output any additional comments.


### Backend Test

In [18]:
# prompt components
prompt_txt = read_txt('backend_test.txt')
backend_test_schema_1 = ResponseSchema(name="revision_needed",
                                        description="Based on your evaluation, are there any errors that need to be fixed? Answer True if yes, False if no.")
backend_test_schema_2 = ResponseSchema(name="feedback",
                                        description="Report with your analysis of the code.")

backend_code = "No code"
backend_logs = "No errors"

# define ouput parser
response_schemas = [backend_test_schema_1, backend_test_schema_2]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
backend_test_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 backend_code = backend_code,
                                 backend_logs = backend_logs)

In [19]:
backend_test_prompt_str = backend_test_prompt[0].content
print(backend_test_prompt_str)

You are a software developer specialized in testing backend systems and it is your responsibility to check code for errors or mistakes. You will be provided with some code that is used to interact with a database as well as some error logs if there are any. 
This is the code you have to check:
No code

These are the error logs:
No errors

Based on this code write a report and point out any mistakes or errors you have found. Explain them and suggest how they could be corrected. Only point errors, mistakes or things that are incorrect and do not talk about things that are well done. It is possible that there are no errors or bad practices in which case your report will be empty.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"revision_needed": string  // Based on your evaluation, are there any errors that need to be fixed? Answer True if yes, False if no.
	"feedback": string  // Report wi

### Backend Doc

In [20]:
# prompt components
prompt_txt = read_txt('backend_doc.txt')
backend_doc_schema_1 = ResponseSchema(name="documentation",
                                        description="The documentation of the code.")

backend_code = "No code"

# define ouput parser
response_schemas = [database_doc_schema_1]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
backend_doc_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 backend_code = backend_code)

In [21]:
backend_doc_prompt_str = backend_doc_prompt[0].content
print(backend_doc_prompt_str)

You are a specialist for writing a documentation for code. It is your responsibility to analyze some given code and write a short description, so that someone else can understand what functionalities there are. You will be provided with some code that is used to access a database. It is your task to write a short documentation that describes what functions there are and what they are capable of. Explicitly point out what URL endpoints there are.
This is the code:
No code
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"documentation": string  // The documentation of the code.
}
```


## **Frontend**

### Frontend Dev

In [22]:
# prompt components
prompt_txt = read_txt('frontend_dev.txt')
frontend_dev_schema_1 = ResponseSchema(name="html_code",
                             description="The html code you wrote.")
frontend_dev_schema_2 = ResponseSchema(name="css_code",
                             description="The css code you wrote.")
frontend_dev_schema_3 = ResponseSchema(name="js_code",
                             description="The javascript code you wrote.")

orchestrator_fe = "Tasks für das Frontend"
backend_doc = "Dokumentation des Backend"

# define ouput parser
response_schemas = [frontend_dev_schema_1, frontend_dev_schema_2, frontend_dev_schema_3]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
frontend_dev_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 orchestrator_fe = orchestrator_fe,
                                 backend_doc = backend_doc)

In [23]:
frontend_dev_prompt_str = frontend_dev_prompt[0].content
print(frontend_dev_prompt_str)

You are a software developer specialized in frontend webpages. It is your responsibility to write the necessary code to display a webpage and perform certain funtionalities. You will be provided with some requirements by your manager and a description of the backend system. It is now your task to provide the html, css and javascript code that fufill the listed requirments.
Tasks für das Frontend

This is the description of the backend functionalities and endpoints you should use:
Dokumentation des Backend

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"html_code": string  // The html code you wrote.
	"css_code": string  // The css code you wrote.
	"js_code": string  // The javascript code you wrote.
}
```
Only output this, and put your generated code in the json. Do not output any additional comments.


### Frontend Test

In [24]:
# prompt components
prompt_txt = read_txt('frontend_test.txt')
frontend_test_schema_1 = ResponseSchema(name="revision_needed",
                                        description="Based on your evaluation, are there any errors that need to be fixed? Answer True if yes, False if no.")
frontend_test_schema_2 = ResponseSchema(name="feedback",
                                        description="Report with your analysis of the code.")

frontend_code = "No code"
frontend_logs = "No errors"

# define ouput parser
response_schemas = [frontend_test_schema_1, frontend_test_schema_2]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
frontend_test_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 frontend_code = frontend_code,
                                 frontend_logs = frontend_logs)

In [25]:
frontend_test_prompt_str = frontend_test_prompt[0].content
print(frontend_test_prompt_str)

You are a software developer specialized in testing webpages and it is your responsibility to check code for errors or mistakes. You will be provided with some code that is used to create a simple webpage as well as some error logs if there are any. 
This is the code you have to check:
No code

These are the error logs:
No errors

Based on this code write a report and point out any mistakes or errors you have found. Explain them and suggest how they could be corrected. Only point errors, mistakes or things that are incorrect do not talk about things that are well done. It is possible that there are no errors or bad practices in this case the report is empty.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"revision_needed": string  // Based on your evaluation, are there any errors that need to be fixed? Answer True if yes, False if no.
	"feedback": string  // Report with your analysis o

### Frontend Doc

In [26]:
# prompt components
prompt_txt = read_txt('frontend_doc.txt')
frontend_doc_schema_1 = ResponseSchema(name="documentation",
                                        description="The documentation of the code.")

frontend_code = "No code"

# define ouput parser
response_schemas = [frontend_doc_schema_1]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
frontend_doc_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 frontend_code = frontend_code)

In [27]:
frontend_doc_prompt_str = frontend_doc_prompt[0].content
print(frontend_doc_prompt_str)

You are a specialist for writing a documentation for code. It is your responsibility to analyze some given code and write a short description, so that someone else can understand what functionalities there are. You will be provided with some code that is used to create a webpage. It is your task to write a short documentation that describes what functions there are and what they are capable of.
This is the code:
No code

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"documentation": string  // The documentation of the code.
}
```


### More stuff

In [2]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

In [4]:
prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template("self.character"),
                MessagesPlaceholder(variable_name="chat_history"),
                HumanMessagePromptTemplate.from_template("{message}"),
            ]
        )

In [7]:
prompt

ChatPromptTemplate(input_variables=['chat_history', 'message'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='self.character')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['message'], template='{message}'))])

In [8]:
def extract_section(input_string, section_type):
    """
    Extracts a section from the input string that starts with ```<section_type>
    and ends with ```

    :param input_string: The string to parse
    :param section_type: The type of section to extract (e.g., 'json', 'python')
    :return: The extracted section or an empty string if not found
    """
    start_marker = f"```{section_type}"
    end_marker = "```"

    start_index = input_string.find(start_marker)
    if start_index == -1:
        return ""  # Start marker not found

    # Adjust start index to skip the marker itself
    start_index += len(start_marker)

    end_index = input_string.find(end_marker, start_index)
    if end_index == -1:
        return ""  # End marker not found

    return input_string[start_index:end_index].strip()

# Example usage:
input_str = "Here is some text ```json\n{\"key\": \"value\"}\n```\nAnd some more text."
section = extract_section(input_str, "json")
print(section)


{"key": "value"}


In [17]:
import re

def extract_code(input_string, language):
    """
    Extracts a code section from the input string that starts with ```<language>
    and ends with ```.
    
    :param input_string: The string to be parsed.
    :param language: The language specifier (e.g. 'json', 'python').
    :return: Extracted code section or an empty string if not found.
    """
    pattern = f"```{language}(.*?)```"
    match = re.search(pattern, input_string, re.DOTALL)

    if match:
        return match.group(1).strip()
    else:
        return ""
    

# Example usage:
input_str = "Here is some text ```json\n{\"key\": \"value\"}\n```\nAnd some more text."
section = extract_code(input_str, "json")
print(section)

{"key": "value"}


In [18]:
# Example usage:
input_str = """Database Tester: The structure of the "subscribers" entity seems to be well-defined, with attributes for "id", "name", and "age" for each subscriber. However, it would be beneficial to include additional attributes such as "email", "subscription_date", or any other relevant information that might be associated with subscribers. This will provide a more comprehensive view of the subscribers and their details.
Database Dev: ```json
{
  "subscribers": [
    {
      "id": 1,
      "name": "John",
      "age": 25,
      "email": "john@example.com",
      "subscription_date": "2022-01-15"
    },
    {
      "id": 2,
      "name": "Alice",
      "age": 30,
      "email": "alice@example.com",
      "subscription_date": "2022-02-20"
    }
  ]
}
```"""
section = extract_code(input_str, "json")
print(section)

{
  "subscribers": [
    {
      "id": 1,
      "name": "John",
      "age": 25,
      "email": "john@example.com",
      "subscription_date": "2022-01-15"
    },
    {
      "id": 2,
      "name": "Alice",
      "age": 30,
      "email": "alice@example.com",
      "subscription_date": "2022-02-20"
    }
  ]
}


In [ ]:
'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```sql" and "```":'

In [14]:
def output_format(language: str, code_only: bool) -> str:
    """
    Generates a markdown code snippet with a specified programming language.

    :param language: The language for the code snippet (e.g., 'python', 'javascript').
    :param code_only: A boolean to force that only the code block is outputed.
    :return: A string containing the formatted markdown code snippet.
    """
    instructions = f'The output should be a markdown code snippet, starting with "```{language}" and ending with "```".'
    if code_only:
        force_code = 'Only output this markdown code snippet. Do not output any additional comments.'
        return instructions+' '+force_code

    return instructions

In [15]:
output_format('sql', True)

'The output should be a markdown code snippet, starting with "```sql" and ending with "```". Only output this markdown code snippet. Do not output any additional comments.'